<a href="https://colab.research.google.com/github/tienhuynh96/NMT_Neural-Machine-Translation/blob/main/%5BDemo%5D_NMT_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**EN-VI Machine Translation using Transformer Model**

##**Dataset**

In [ ]:
!pip install -q datasets sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00


##**1.1 Load dataset from dataset library**

The IWSLT'15 English-Vietnamese data is used from Stanford NLP group.

In [ ]:
from datasets import load_dataset

data = load_dataset(
    "mt_eng_vietnamese",
    "iwslt2015-en-vi"
)

In [ ]:
data['train']

Dataset({
    features: ['translation'],
    num_rows: 133318
})

In [ ]:
data['train']['translation'][0]

{'en': 'Rachel Pike : The science behind a climate headline',
 'vi': 'Khoa học đằng sau một tiêu đề về khí hậu'}

##**1.2 Incase can not load dataset from dataset**

Wget data from github: https://github.com/stefan-it/nmt-en-vi

In [ ]:
# Incase can not load dataset from dataset
# Try
# Wget data from github: https://github.com/stefan-it/nmt-en-vi
# Train file
!wget "https://github.com/stefan-it/nmt-en-vi/raw/master/data/train-en-vi.tgz"
# Dev file
!wget "https://github.com/stefan-it/nmt-en-vi/raw/master/data/dev-2012-en-vi.tgz"
# Test file
!wget "https://github.com/stefan-it/nmt-en-vi/raw/master/data/test-2013-en-vi.tgz"

--2024-05-29 07:43:58--  https://github.com/stefan-it/nmt-en-vi/raw/master/data/train-en-vi.tgz
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/stefan-it/nmt-en-vi/master/data/train-en-vi.tgz [following]
--2024-05-29 07:43:58--  https://raw.githubusercontent.com/stefan-it/nmt-en-vi/master/data/train-en-vi.tgz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9903559 (9.4M) [application/octet-stream]
Saving to: ‘train-en-vi.tgz’

train-en-vi.tgz     100%[===================>]   9.44M  --.-KB/s    in 0.07s   

2024-05-29 07:43:59 (137 MB/s) - ‘train-en-vi.tgz’ saved [9903559/9903559]

--2024-05-29 0

In [ ]:
# extract the tgz file
!tar -xzf train-en-vi.tgz
!tar -xzf dev-2012-en-vi.tgz
!tar -xzf test-2013-en-vi.tgz

In [ ]:
# Create a dataset shape similar with datashape which download from datasets library
data = {
    "train": {
                "translation":[]
              },
    "validation": {
                "translation":[]
              },
    "test": {
                "translation":[]
              },
}

In [ ]:
data

{'train': {'translation': []},
 'validation': {'translation': []},
 'test': {'translation': []}}

In [ ]:
# Add path for train, test, validation file
# Train file
train_source_file = "/content/train.en"
train_target_file = "/content/train.vi"
# Validation file
val_source_file = "/content/tst2012.en"
val_target_file = "/content/tst2012.vi"
# Test file
test_source_file = "/content/tst2013.en"
test_target_file = "/content/tst2013.vi"

In [ ]:
# Build function: Get examples from sourse, target file and append into data
def generate_examples(source_file, target_file, data, dataname):
    # Open and read source file and target file of "train, test, validation" data
    with open(source_file, encoding="utf-8") as f:
        source_sentences = f.read().split("\n")
    with open(target_file, encoding="utf-8") as f:
        target_sentences = f.read().split("\n")

    # Add examples of "train, test, validation" data
    data_gen = data[dataname]['translation']
    source, target = "en", "vi"
    for idx, (l1, l2) in enumerate(zip(source_sentences, target_sentences)):
        result = {source: l1, target: l2}
        data_gen.append(result)
    return data

In [ ]:
data = generate_examples(source_file=train_source_file, target_file=train_target_file,
                        data=data, dataname="train")

data = generate_examples(test_source_file, test_target_file, data, "test")
data = generate_examples(val_source_file, val_target_file, data, "validation")

In [ ]:
data['train']['translation'][0]

{'en': 'Rachel Pike : The science behind a climate headline',
 'vi': 'Khoa học đằng sau một tiêu đề về khí hậu'}

In [ ]:
data['test']['translation'][0]

{'en': 'When I was little , I thought my country was the best on the planet , and I grew up singing a song called &quot; Nothing To Envy . &quot;',
 'vi': 'Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên là đất nước tốt nhất trên thế giới và tôi thường hát bài &quot; Chúng ta chẳng có gì phải ghen tị . &quot;'}

In [ ]:
data['validation']['translation'][0]

{'en': 'How can I speak in 10 minutes about the bonds of women over three generations , about how the astonishing strength of those bonds took hold in the life of a four-year-old girl huddled with her young sister , her mother and her grandmother for five days and nights in a small boat in the China Sea more than 30 years ago , bonds that took hold in the life of that small girl and never let go -- that small girl now living in San Francisco and speaking to you today ?',
 'vi': 'Làm sao tôi có thể trình bày trong 10 phút về sợi dây liên kết những người phụ nữ qua ba thế hệ , về việc làm thế nào những sợi dây mạnh mẽ đáng kinh ngạc ấy đã níu chặt lấy cuộc sống của một cô bé bốn tuổi co quắp với đứa em gái nhỏ của cô bé , với mẹ và bà trong suốt năm ngày đêm trên con thuyền nhỏ lênh đênh trên Biển Đông hơn 30 năm trước , những sợi dây liên kết đã níu lấy cuộc đời cô bé ấy và không bao giờ rời đi -- cô bé ấy giờ sống ở San Francisco và đang nói chuyện với các bạn hôm nay ?'}

##**Tokenization**

In [ ]:
# Get_tokenizer and build vocab
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated

In [ ]:
# Set source and target language
SRC_LANGUAGE = 'en'
TGT_LANGUAGE = 'vi'

# Create token_transform and vocab_transform is empty dictionary
token_transform = {}
vocab_transform = {}

# Add tokenizer for source and target language
token_transform[SRC_LANGUAGE] = get_tokenizer('basic_english')
token_transform[TGT_LANGUAGE] = get_tokenizer('basic_english')

# Set special symbols and its id
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>','eos']

In [ ]:
# Build vocab
# Build yield tokens function: input data_iter and language (en or vi)
def yield_tokens(data_iter, lang):
  for data_sample in data_iter['translation']:
    yield token_transform[lang](data_sample[lang])

# Build vocab for source and target language
for lang in [SRC_LANGUAGE, TGT_LANGUAGE]:
  # Get data from train data
  train_iter = data['train']

  # Create torchtext's vocab object
  vocab_transform[lang] = build_vocab_from_iterator(
      yield_tokens(train_iter, lang),   # yield for each language
      min_freq=1,               # Minimum of appear tokens in vocab is stored
      specials=special_symbols, # Get special symbols
      special_first=True        # Set special symbol ids is first
  )

  # Set set_default_index is UNK_IDX
  vocab_transform[lang].set_default_index(UNK_IDX)

In [ ]:
# Build vocab for source language
vocab_transform[SRC_LANGUAGE].get_itos()[:10]

['<unk>', '<pad>', '<bos>', 'eos', ',', '.', 'the', 'and', 'to', '&apos']

In [ ]:
# Build vocab for target language
vocab_transform[TGT_LANGUAGE].get_itos()[:10]

['<unk>', '<pad>', '<bos>', 'eos', ',', '.', 'và', 'tôi', 'là', 'một']

In [ ]:
len(vocab_transform[SRC_LANGUAGE]), len(vocab_transform[TGT_LANGUAGE])

(47270, 21113)

##**Dataloader**

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence     # padding follow the highest sequence length in dataset

# Set max len
MAX_LEN = 100

# helper function to club together sequential operations
# Take a series of transformation functions as arguments and returns a new function
def sequential_transforms(*transforms):   # *transforms allows the function to accept any number of transformation functions as arguments.
    def func(txt_input): # inner function
        # Applying Transformations in Sequence
        # For each transformation, it applies the function to txt_input, updating txt_input with the transformed result.
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func


# function to add BOS/EOS and create tensor for input sequence indices: BOS-ids-EOS
def tensor_transform(token_ids):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# Function to truncate
def truncate(sample):
    if sample.size(0) > MAX_LEN:
        return sample[:MAX_LEN, :]
    else:
        return sample

In [ ]:
# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
text_transform = {}
for lang in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[lang] = sequential_transforms(
        token_transform[lang], # Tokenization   => tokens
        vocab_transform[lang], # Numericalization => ids
        tensor_transform # Add BOS/EOS and create tensor  => bos ids eos
    )

# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    # transforms for each sample in batch
    for sample in batch:
        src_sample, tgt_sample = sample[SRC_LANGUAGE], sample[TGT_LANGUAGE]
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample).to(dtype=torch.int64))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample).to(dtype=torch.int64))

    # Add pad sequence and truncate
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX, batch_first=True)
    src_batch = truncate(src_batch)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX, batch_first=True)
    tgt_batch = truncate(tgt_batch)
    return src_batch, tgt_batch

In [ ]:
from torch.utils.data import DataLoader

# Set batch size
BATCH_SIZE = 32

# Build dataloader for train, validation, test data
train_dataloader = DataLoader(
    data['train']['translation'],
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn
)

valid_dataloader = DataLoader(
    data['validation']['translation'],
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn
)

test_dataloader = DataLoader(
    data['test']['translation'],
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn
)

In [ ]:
# Check the data of first batch
src_ids, tgt_ids = next(iter(train_dataloader))

In [ ]:
# Check shape
src_ids.shape, tgt_ids.shape

(torch.Size([32, 52]), torch.Size([32, 78]))

In [ ]:
src_ids[0]

tensor([    2,  6429, 17576,     6,   295,   553,    11,   682,  5334,     3,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1])

In [ ]:
# Decode the source ids
' '.join([vocab_transform[SRC_LANGUAGE].lookup_token(token) for token in src_ids[0]])

'<bos> rachel pike the science behind a climate headline eos <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

In [ ]:
# Decode the target ids
' '.join([vocab_transform[TGT_LANGUAGE].lookup_token(token) for token in tgt_ids[0]])

'<bos> khoa học đằng sau một tiêu đề về khí hậu eos <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'

##**Model**

In [ ]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    # Add positional encoding to token embedding
    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)



In [ ]:
# Build Model
# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        # Transofrmer is sequence to sequence model
        self.transformer = Transformer(d_model=emb_size,    # Embedding size
                                       nhead=nhead,         # n head
                                       num_encoder_layers=num_encoder_layers,   # Number encoder layers
                                       num_decoder_layers=num_decoder_layers,   # Number decoder layers
                                       dim_feedforward=dim_feedforward,         # Feedforward dimention
                                       dropout=dropout,                         # Dropout
                                       batch_first=True)
        # generator is linear function
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        # Token embedding for source token
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        # Token embedding for target token
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        # Add position for encode and decode
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,       # For seperate the token and padding
                tgt_mask: Tensor,       # For seperate the token and padding
                src_padding_mask: Tensor,   # Mask the token each time step to train
                tgt_padding_mask: Tensor,   # Mask the token each time step to train
                memory_key_padding_mask: Tensor):
        # Compute src_emb: 1- compute src_tok_emb (source token embedding), 2 - compute positional_encoding
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        # Compute tgt_emb: 1- compute tgt_tok_emb (target token embedding), 2 - compute positional_encoding
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        # Compute transformer output
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)      # B x S X H
        # Generator in vocab size
        return self.generator(outs)         # B X S X target vocab size

    # Create encoder
    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    # Create decoder
    # Memory is output state of encoder
    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [ ]:
(torch.triu(torch.ones((6, 6), device=DEVICE))  == 1).transpose(0, 1)

tensor([[ True, False, False, False, False, False],
        [ True,  True, False, False, False, False],
        [ True,  True,  True, False, False, False],
        [ True,  True,  True,  True, False, False],
        [ True,  True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True,  True]])

In [ ]:
torch.zeros((6, 6),device=DEVICE).type(torch.bool)

tensor([[False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False]])

In [ ]:
# Build function to create mask matrix,
def generate_square_subsequent_mask(sz):
    # Create one matrix => triu matrix => compare with 1 => transpose to have type and shape of mask token
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    # Fill: 0 value -> -inf; 1 value -> 0:
    # if value is -inf, when softmax we get the value 0
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

# Build function to create mask for target and source input
def create_mask(src, tgt):
    # Get len of sequence in source
    src_seq_len = src.shape[1]
    # Get len of sequence in target
    tgt_seq_len = tgt.shape[1]

    # Create target mask
    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    # Create source mask: a zero matrix or source do not need mask
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    # Create source padding mask to figure out where is token and padding
    src_padding_mask = (src == PAD_IDX)
    # Create target padding mask to figure out where is token and padding
    tgt_padding_mask = (tgt == PAD_IDX)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [ ]:
# Add parameter for model
# Add parameter values
SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

# Create transformer model and put to device
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)
transformer = transformer.to(DEVICE)

# Create loss
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

# Create otimizer
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [ ]:
# Test model
# Get values of the first batch and put to device
src_ids, tgt_ids = next(iter(train_dataloader))
src_ids = src_ids.to(DEVICE)
tgt_ids = tgt_ids.to(DEVICE)
# Get target input
tgt_input = tgt_ids[:, :-1]
# Get target output
tgt_output = tgt_ids[:, 1:]
# create mask
src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src_ids, tgt_input)
# Compute model: logits # B X S X target vocab size
logits = transformer(
    src_ids, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask
)
# Compute loss
loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_output.reshape(-1))

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5137: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


In [ ]:
logits.shape

torch.Size([32, 77, 21113])

In [ ]:
loss

tensor(10.0856, grad_fn=<NllLossBackward0>)

##**Trainer**

In [ ]:
import time
from tqdm import tqdm

# Creaete train epoch function
def train_epoch(model, optimizer, criterion, train_dataloader, device):
    #  train mode
    model.train()
    losses = []

    # Iterator to get data from train_loader
    for src_ids, tgt_ids in train_dataloader:
        # put src_ids, tgt_ids to device
        src_ids = src_ids.to(device)
        tgt_ids = tgt_ids.to(device)

        # Get input and output target
        tgt_input = tgt_ids[:, :-1]
        tgt_output = tgt_ids[:, 1:]

        # Create mask
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src_ids, tgt_input)
        # Compute model: # B X S X target vocab size
        output = model(
                src_ids, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        # Set optimizer
        optimizer.zero_grad()

        # Compute loss
        loss = criterion(
            output.reshape(-1, output.shape[-1]),
            tgt_output.reshape(-1))

        # backward
        loss.backward()

        # Update weight
        optimizer.step()
        # save loss
        losses.append(loss.item())

    return sum(losses) / len(losses)

# Compute loss
def evaluate(model, data_loader, criterion, device):
    # eval mode
    model.eval()
    losses = []
    with torch.no_grad():
        for src_ids, tgt_ids in data_loader:
            # put src_ids, tgt_ids to device
            src_ids = src_ids.to(device)
            tgt_ids = tgt_ids.to(device)

            # Get input and output target
            tgt_input = tgt_ids[:, :-1]
            tgt_output = tgt_ids[:, 1:]

            # Create mask
            src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src_ids, tgt_input)
            # Compute output
            output = model(
                src_ids, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask
            )
            # Compute loss
            loss = criterion(
                output.reshape(-1, output.shape[-1]),
                tgt_output.reshape(-1)
            )
            # Save loss
            losses.append(loss.item())
    return sum(losses) / len(losses)

# Creaete train function
def train(model, train_dataloader, valid_dataloader, optimizer, criterion, device, epochs):
    # Iterator for each epoch
    for epoch in tqdm(range(1, epochs+1)):
        # Set start time
        start_time = time.time()
        # Compute train loss
        train_loss = train_epoch(model, optimizer, criterion, train_dataloader, device)
        # Compute validation loss
        valid_loss = evaluate(model, valid_dataloader, criterion, device)
        # Set end time
        end_time = time.time()
        print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {valid_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

##**Training**

In [ ]:
# Create parameter input
SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

# Create model and put to device
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)
transformer = transformer.to(DEVICE)

# Set criterion
criterion = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
# Set optimizer
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
# Set epoch
epochs = 1
# train model
train(transformer, train_dataloader, valid_dataloader, optimizer, criterion, DEVICE, epochs)

  0%|          | 0/1 [00:00<?, ?it/s]

##**Inference**

In [ ]:
# Build greedy decode function
# generate next token base on previous predicted tokens
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    # Put source to device
    src = src.to(DEVICE)
    # Put source mask to device
    src_mask = src_mask.to(DEVICE)

    # Compute out put of encoder (get hidden state)
    memory = model.encode(src, src_mask)
    # Create start tokens as start symbol
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)

    # Iterator to generate next token untill meet EOS_IDS or max_len
    for i in range(max_len-1):
        # Put memory to device
        memory = memory.to(DEVICE)
        # Create target mask
        tgt_mask = (generate_square_subsequent_mask(ys.size(1))
                    .type(torch.bool)).to(DEVICE)
        # Compute out put of decoder
        out = model.decode(ys, memory, tgt_mask)  # B, S, H
        # transpose, to get squence (B, S) => (S, B)
        out = out.transpose(0, 1)
        # Compute generator
        prob = model.generator(out[:, -1])
        # Get the index of word
        _, next_word = torch.max(prob, dim=1)
        # Get next word
        next_word = next_word[-1].item() # index

        # Update start input token index
        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
        # If the next word is EOS IDS, break
        if next_word == EOS_IDX:  #ESO : 3
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    # test transform for source sentence
    src = text_transform[SRC_LANGUAGE](src_sentence).view(1, -1)
    # number of tokens in source sentence
    num_tokens = src.shape[1]
    # Create mask
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    # Generate next token
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(
        vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [ ]:
# test function
translate(transformer, "i go to school")

In [ ]:
# Compute metric: sacrebleu
from tqdm import tqdm
import sacrebleu

pred_sentences, tgt_sentences = [], []
for sample in tqdm(data['test']['translation']):
    # Get data from test
    src_sentence = sample[SRC_LANGUAGE]
    tgt_sentence = sample[TGT_LANGUAGE]

    # translate
    pred_sentence = translate(transformer, src_sentence)
    pred_sentences.append(pred_sentence)

    tgt_sentences.append(tgt_sentence)
# Compute metric
bleu_score = sacrebleu.corpus_bleu(pred_sentences, [tgt_sentences], force=True)
bleu_score

100%|██████████| 1269/1269 [01:53<00:00, 11.14it/s]


BLEU = 6.91 46.8/16.8/6.4/2.4 (BP = 0.659 ratio = 0.706 hyp_len = 23819 ref_len = 33738)